In [1]:
import sys
import os

sys.path.append("/data/igor/loss_traces")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from results.result_processing import get_trace_reduction
from results.utils import (
    make_precision_recall_at_k_df,
    get_precision_at_fpr,
    make_precision_recall_at_k_df_single_threshold,
)
from results.final_model_metrics import get_final_model_metrics
from attacks import AttackConfig, LiRAAttack
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
import pickle
import opacus
from opacus.validators import ModuleValidator

In [ ]:
target_id = "target"
attack = "LiRA"
augment = True
batchsize = 32
num_workers = 8
gpu = ":1"
dataset = "CIFAR10"
arch = "wrn28-2"

all_dfs = {}
for clipnorm in ("10", "3", "1"):
    for noise_multiplier in ("0", "0.1", "1"):
        exp_id = f"{arch}_{dataset}_clipnorm_{clipnorm}_nm_{noise_multiplier}"

        csv_path = f'data/trace_df_{arch}_{dataset}_clip{clipnorm}_nm_{noise_multiplier}.csv'
        if os.path.exists(csv_path):
            all_dfs[(clipnorm, noise_multiplier)] = pd.read_csv(csv_path, index_col=0)
            continue

        config = AttackConfig(
            exp_id=exp_id,
            target_id=target_id,
            checkpoint=None,
            arch=arch,
            dataset=dataset,
            attack=attack,
            augment=augment,
            batchsize=batchsize,
            num_workers=num_workers,
            gpu=gpu,
            is_dp=True,
        )
        attack = LiRAAttack(config)
        results = attack.run()

        df = get_final_model_metrics(
            model=attack.model, data_loader=attack.attack_loaders[0]
        )
        df = pd.merge(df, results, left_index=True, right_index=True)

        trace = get_trace_reduction(exp_id, reduction="iqr")
        trace_df = pd.DataFrame({'trace_value': trace})
        trace_df = pd.merge(trace_df, df, left_index=True, right_index=True)

        trace_df.to_csv(csv_path)
        all_dfs[(clipnorm, noise_multiplier)] = trace_df


Attack AUC: 0.6395701872


  0%|                                                                                                                                                                                                                       | 0/1563 [00:00<?, ?it/s]/home/igor/.venv/lib/python3.12/site-packages/captum/attr/_core/deep_lift.py:810: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  attributions = super().attribute.__wrapped__(  # type: ignore
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [16:25<00:00,  1.59it/s]


No intermediate results found at /data/igor/data/scaled_logits_intermediate/wrn28-2_CIFAR10_clipnorm_10_nm_0.1.pt. Computing intermediate results...
Computing all metrics...
Computed metrics for shadow 0
Computed metrics for shadow 1
Computed metrics for shadow 2
Computed metrics for shadow 3
Computed metrics for shadow 4
Computed metrics for shadow 5
Computed metrics for shadow 6
Computed metrics for shadow 7
Computed metrics for shadow 8
Computed metrics for shadow 9
Computed metrics for shadow 10
Computed metrics for shadow 11
Computed metrics for shadow 12
Computed metrics for shadow 13
Computed metrics for shadow 14
Computed metrics for shadow 15
Computing statistics for losses...
Computing statistics for logits...
Computing statistics for scaled_logits...
Computing statistics for target_logits...
Performance Timings:
Total computation time: 366.87s
Finished stats
Attack AUC: 0.5111000800000001


  0%|                                                                                                                                                                                                                       | 0/1563 [00:00<?, ?it/s]/home/igor/.venv/lib/python3.12/site-packages/captum/attr/_core/deep_lift.py:810: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  attributions = super().attribute.__wrapped__(  # type: ignore
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [16:24<00:00,  1.59it/s]


No intermediate results found at /data/igor/data/scaled_logits_intermediate/wrn28-2_CIFAR10_clipnorm_10_nm_1.pt. Computing intermediate results...
Computing all metrics...
Computed metrics for shadow 0
Computed metrics for shadow 1
Computed metrics for shadow 2
Computed metrics for shadow 3
Computed metrics for shadow 4
Computed metrics for shadow 5
Computed metrics for shadow 6
Computed metrics for shadow 7
Computed metrics for shadow 8
Computed metrics for shadow 9
Computed metrics for shadow 10
Computed metrics for shadow 11
Computed metrics for shadow 12
Computed metrics for shadow 13
Computed metrics for shadow 14
Computed metrics for shadow 15
Computing statistics for losses...
Computing statistics for logits...
Computing statistics for scaled_logits...
Computing statistics for target_logits...
Performance Timings:
Total computation time: 369.37s
Finished stats
Attack AUC: 0.49927176479999996


...

  0%|                                                                                                                                                                                                                       | 0/1563 [00:00<?, ?it/s]/home/igor/.venv/lib/python3.12/site-packages/captum/attr/_core/deep_lift.py:810: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  attributions = super().attribute.__wrapped__(  # type: ignore
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [16:23<00:00,  1.59it/s]


No intermediate results found at /data/igor/data/scaled_logits_intermediate/wrn28-2_CIFAR10_clipnorm_3_nm_0.1.pt. Computing intermediate results...
Computing all metrics...
Computed metrics for shadow 0
Computed metrics for shadow 1
Computed metrics for shadow 2
Computed metrics for shadow 3
Computed metrics for shadow 4
Computed metrics for shadow 5
Computed metrics for shadow 6
Computed metrics for shadow 7
Computed metrics for shadow 8
Computed metrics for shadow 9
Computed metrics for shadow 10
Computed metrics for shadow 11
Computed metrics for shadow 12
Computed metrics for shadow 13
Computed metrics for shadow 14
Computed metrics for shadow 15
Computing statistics for losses...
Computing statistics for logits...
Computing statistics for scaled_logits...
Computing statistics for target_logits...
Performance Timings:
Total computation time: 370.16s
Finished stats
Attack AUC: 0.5154872895999999


  0%|                                                                                                                                                                                                                       | 0/1563 [00:00<?, ?it/s]/home/igor/.venv/lib/python3.12/site-packages/captum/attr/_core/deep_lift.py:810: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  attributions = super().attribute.__wrapped__(  # type: ignore
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [16:29<00:00,  1.58it/s]


No intermediate results found at /data/igor/data/scaled_logits_intermediate/wrn28-2_CIFAR10_clipnorm_3_nm_1.pt. Computing intermediate results...
Computing all metrics...
Computed metrics for shadow 0
Computed metrics for shadow 1
Computed metrics for shadow 2
Computed metrics for shadow 3
Computed metrics for shadow 4
Computed metrics for shadow 5
Computed metrics for shadow 6
Computed metrics for shadow 7
Computed metrics for shadow 8
Computed metrics for shadow 9
Computed metrics for shadow 10
Computed metrics for shadow 11
Computed metrics for shadow 12
Computed metrics for shadow 13
Computed metrics for shadow 14
Computed metrics for shadow 15
Computing statistics for losses...
Computing statistics for logits...
Computing statistics for scaled_logits...
Computing statistics for target_logits...
Performance Timings:
Total computation time: 375.26s
Finished stats
Attack AUC: 0.4996302704


  0%|                                                                                                                                                                                                                       | 0/1563 [00:00<?, ?it/s]/home/igor/.venv/lib/python3.12/site-packages/captum/attr/_core/deep_lift.py:810: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  attributions = super().attribute.__wrapped__(  # type: ignore
 53%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                               | 836/1563 [08:44<08:35,  1.41it/s]

In [51]:
results = attack.run()

No intermediate results found at /data/igor/data/scaled_logits_intermediate/wrn28-2_CIFAR10_clipnorm_1_nm_0.pt. Computing intermediate results...
Computing all metrics...
Computed metrics for shadow 0
Computed metrics for shadow 1
Computed metrics for shadow 2
Computed metrics for shadow 3
Computed metrics for shadow 4
Computed metrics for shadow 5
Computed metrics for shadow 6
Computed metrics for shadow 7
Computed metrics for shadow 8
Computed metrics for shadow 9
Computed metrics for shadow 10
Computed metrics for shadow 11
Computed metrics for shadow 12
Computed metrics for shadow 13
Computed metrics for shadow 14
Computed metrics for shadow 15
Computing statistics for losses...
Computing statistics for logits...
Computing statistics for scaled_logits...
Computing statistics for target_logits...
Performance Timings:
Total computation time: 355.00s
Finished stats
Attack AUC: 0.5354067664


In [40]:
df = get_final_model_metrics(
    model=attack.model, data_loader=attack.attack_loaders[0], metrics=["loss"]
)
df = pd.merge(df, results, left_index=True, right_index=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:10<00:00, 150.90it/s]


In [41]:
trace = get_trace_reduction(exp_id, reduction="iqr")
trace_df = pd.DataFrame({'trace_value': trace})
trace_df = pd.merge(trace_df, df, left_index=True, right_index=True)

In [49]:
make_precision_recall_at_k_df_single_threshold(
    scores_df=trace_df,
    ground_truth_df=trace_df,
    fpr_threshold=0.01,
    k_frac=0.05,
)

{'spearman_trace_value': 0.13480467888530726,
 'spearman_loss': 0.02813197981283565,
 'spearman_confidence': 0.005641580617373862,
 'spearman_lira_score': 1.0,
 'fpr': 0.01004,
 'tpr': 0.03556,
 'positives': 889,
 'precision_random_guess': 0.03556,
 'recall_random_guess': 0.05,
 'overlap_trace_value_desc': 192,
 'precision_trace_value_desc': 0.1536,
 'recall_trace_value_desc': 0.21597300337457817,
 'overlap_trace_value_asc': 0,
 'precision_trace_value_asc': 0.0,
 'recall_trace_value_asc': 0.0,
 'overlap_loss_desc': 10,
 'precision_loss_desc': 0.008,
 'recall_loss_desc': 0.01124859392575928,
 'overlap_loss_asc': 1,
 'precision_loss_asc': 0.0008,
 'recall_loss_asc': 0.0011248593925759281,
 'overlap_confidence_desc': 1,
 'precision_confidence_desc': 0.0008,
 'recall_confidence_desc': 0.0011248593925759281,
 'overlap_confidence_asc': 67,
 'precision_confidence_asc': 0.0536,
 'recall_confidence_asc': 0.07536557930258718,
 'overlap_lira_score_desc': 889,
 'precision_lira_score_desc': 0.7112,